# Fine-tune Training VGG16 on CIFAR10
- In this notebook, we will train the CIFAR-10 data set on VGG16
- The problem is that the pre-trained model of VGG16 was for ImageNet not CIFAR10
- This is an attempt to investigate and see how much accuracy VGG16 can achieve on this
- Also note that this is fine-tune training. Which means we will re-tune the pre-trained weights.
- **WARNING: Better if you have a GPU installed**

# Importing Packages and Setting of CUDA

In [2]:
# This is to manually control which GPUs to use
# In the laboratory we have a server with 4 GPUs
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

import torch

print(torch.cuda.device_count())  # Should output 2
print(torch.cuda.get_device_name(0))  # Should correspond to GPU 2
print(torch.cuda.get_device_name(1))  # Should correspond to GPU 3

import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm import tqdm

# Set device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Main Compute: {device}")

2
NVIDIA GeForce RTX 2080 Ti
NVIDIA GeForce RTX 2080 Ti
Main Compute: cuda:0


# Downloading and Preparing CIFAR10 Dataset

In [3]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet standards
                         std=[0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64,
                                           shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64,
                                          shuffle=False, num_workers=2)

# Load and Modify VGG16 Model

In [4]:
# Load pretrained VGG16 model
model = models.vgg16(pretrained=True)

# Freeze feature parameters
for param in model.features.parameters():
    param.requires_grad = False

# Modify the classifier
model.classifier[6] = nn.Linear(4096, 10)

# Move model to the appropriate device
# Utilizes multiple GPUs
model = torch.nn.DataParallel(model)
model.to(device)

/users/micas/rantonio/anaconda3/envs/tensorx/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/micas/rantonio/anaconda3/envs/tensorx/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DataParallel(
  (module): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      

# Define Loss Function and Optimizer

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.module.classifier.parameters(), lr=0.001)

# Retrain the Model
- Note that we can increase epoch

In [7]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch 1/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 782/782 [02:53<00:00,  4.51it/s]


Epoch [1/10], Loss: 0.7115


Epoch 2/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 782/782 [02:51<00:00,  4.56it/s]


Epoch [2/10], Loss: 0.5214


Epoch 3/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 782/782 [02:52<00:00,  4.53it/s]


...

Epoch [8/10], Loss: 0.2883


Epoch 9/10: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 782/782 [02:53<00:00,  4.51it/s]


Epoch [9/10], Loss: 0.2631


Epoch 10/10: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 782/782 [02:52<00:00,  4.52it/s]

Epoch [10/10], Loss: 0.2462


# Evaulating the Model

In [8]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on CIFAR-10 test images: {100 * correct / total:.2f}%')


Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:23<00:00,  6.58it/s]

Accuracy on CIFAR-10 test images: 87.22%


# Saving the Model
- Note, the model is quite large and therefore needs to be saved somewhere separately
- For this case it's around 500 MB large

In [10]:
# If using DataParallel, access the underlying model
torch.save(model.module.state_dict(), './pretrained_models/vgg16_train_finetune_cifar10.pth')

# Conclusion
- Fine-tuning the parameters allows us to achieve around 87.22% from 11.2%
- Re-training helps a lot!